In [7]:
import pyspark 
from pyspark.sql  import SparkSession 
from pyspark.sql.functions import collect_list, col, lit 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, DateType, ArrayType
import os 
import json 
from pyspark.sql.functions import to_date, when
from urllib.parse import urlparse

In [8]:
aws_access_key = "gROKOCHUyHipWdVYEMoi"
aws_secret_key = "DVAEcJ7qxZNlisV43jZc7bUIEK5rZI7YK6HP7bkl"
aws_s3_endpoint = "http://122.248.249.238:9000" 
aws_s3_region = "us-east-1" 
nessi_warehouse = "s3a://standardize"
nessi_uri = "http://122.248.249.238:19120/api/v1"
raw_csv_path = "s3a://raw/CRMUSER_ACCOUNTS.csv"

In [ ]:
with open("./requirements.txt", "r") as f:
    jar_files = [line.strip() for line in f if line.strip()] 

In [ ]:
import itc_utils.flight_service as itcfs

# Cấu hình dữ liệu yêu cầu
nb_data_request = {
    'data_name': """requirement.txt""",  # Đặt tên cho file bạn muốn đọc
    'interaction_properties': {
        'infer_schema': 'true',  # Cấu hình tự động suy diễn schema
        'infer_as_varchar': 'false'
    }
}

# Lấy thông tin yêu cầu Flight từ dịch vụ
flight_request = itcfs.get_data_request(nb_data_request=nb_data_request)

# Đọc dữ liệu từ file requirements.txt sử dụng Flight Service
sp_df_0 = spark.read.format("com.ibm.connect.spark.flight") \
    .option("flight.location", itcfs.get_flight_service_url()) \
    .option("flight.command", itcfs.get_flight_cmd(data_request=flight_request)) \
    .option("flight.authToken", itcfs.get_bearer_token()) \
    .load()

# Hiển thị 10 dòng dữ liệu từ file txt (nếu có dữ liệu)
sp_df_0.show(10)


In [ ]:
jar_files = sp_df_0.select("jars").rdd.flatMap(lambda x: x).collect()
print(jar_files)

In [ ]:
!wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-aws-bundle/1.5.2/iceberg-aws-bundle-1.5.2.jar
!wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.4_2.12/1.5.0/iceberg-spark-runtime-3.4_2.12-1.5.0.jar
!wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-common/1.4.0/iceberg-common-1.4.0.jar
!wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-core/1.4.0/iceberg-core-1.4.0.jar
!wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-bundled-guava/1.4.0/iceberg-bundled-guava-1.4.0.jar
!wget https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-nessie/1.4.0/iceberg-nessie-1.4.0.jar

# Nessie
!wget https://repo1.maven.org/maven2/org/projectnessie/nessie-integrations/nessie-spark-extensions-3.4_2.12/0.77.1/nessie-spark-extensions-3.4_2.12-0.77.1.jar
!wget https://repo1.maven.org/maven2/org/projectnessie/nessie/nessie-client/0.71.0/nessie-client-0.71.0.jar
!wget https://repo1.maven.org/maven2/org/projectnessie/nessie/nessie-model/0.71.0/nessie-model-0.71.0.jar

# AWS SDK
!wget https://repo1.maven.org/maven2/software/amazon/awssdk/url-connection-client/2.24.8/url-connection-client-2.24.8.jar
!wget https://repo1.maven.org/maven2/software/amazon/awssdk/utils/2.24.8/utils-2.24.8.jar
!wget https://repo1.maven.org/maven2/software/amazon/awssdk/http-client-spi/2.24.8/http-client-spi-2.24.8.jar
!wget https://repo1.maven.org/maven2/software/amazon/awssdk/metrics-spi/2.24.8/metrics-spi-2.24.8.jar
!wget https://repo1.maven.org/maven2/software/amazon/awssdk/annotations/2.24.8/annotations-2.24.8.jar

# Hadoop
!wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar
!wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/3.3.4/hadoop-common-3.3.4.jar
!wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-auth/3.3.4/hadoop-auth-3.3.4.jar
!wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-annotations/3.3.4/hadoop-annotations-3.3.4.jar

# AWS Java SDK
!wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar

# Support Libraries
!wget https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-core/2.15.2/jackson-core-2.15.2.jar
!wget https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.15.2/jackson-databind-2.15.2.jar
!wget https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-annotations/2.15.2/jackson-annotations-2.15.2.jar
!wget https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.36/slf4j-api-1.7.36.jar
!wget https://repo1.maven.org/maven2/org/slf4j/slf4j-reload4j/1.7.36/slf4j-reload4j-1.7.36.jar
!wget https://repo1.maven.org/maven2/org/apache/commons/commons-compress/1.21/commons-compress-1.21.jar
!wget https://repo1.maven.org/maven2/org/apache/avro/avro/1.11.1/avro-1.11.1.jar
!wget https://repo1.maven.org/maven2/io/airlift/aircompressor/0.25/aircompressor-0.25.jar

# Additional Required Dependencies
!wget https://repo1.maven.org/maven2/org/apache/commons/commons-lang3/3.12.0/commons-lang3-3.12.0.jar
!wget https://repo1.maven.org/maven2/commons-codec/commons-codec/1.15/commons-codec-1.15.jar
!wget https://repo1.maven.org/maven2/org/apache/commons/commons-text/1.4/commons-text-1.4.jar
!wget https://repo1.maven.org/maven2/commons-io/commons-io/2.8.0/commons-io-2.8.0.jar
!wget https://repo1.maven.org/maven2/com/google/guava/guava/30.1.1-jre/guava-30.1.1-jre.jar
!wget https://repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.9.47/RoaringBitmap-0.9.47.jar

In [ ]:
# Danh sách các file JAR
jar_files = [
    "jars/iceberg-spark-runtime-3.4_2.12-1.5.0.jar",
    "jars/iceberg-api-1.4.0.jar",
    "jars/iceberg-common-1.4.0.jar",
    "jars/iceberg-core-1.4.0.jar",
    "jars/iceberg-bundled-guava-1.4.0.jar",
    "jars/iceberg-nessie-1.4.0.jar",
    "jars/nessie-spark-extensions-3.4_2.12-0.77.1.jar",
    "jars/nessie-client-0.71.0.jar",
    "jars/nessie-model-0.71.0.jar",
    "jars/bundle-2.24.8.jar",
    "jars/url-connection-client-2.24.8.jar",
    "jars/utils-2.24.8.jar",
    "jars/http-client-spi-2.24.8.jar",
    "jars/metrics-spi-2.24.8.jar",
    "jars/annotations-2.24.8.jar",
    "jars/hadoop-aws-3.3.4.jar",
    "jars/hadoop-common-3.3.4.jar",
    "jars/hadoop-auth-3.3.4.jar",
    "jars/hadoop-annotations-3.3.4.jar",
    "jars/aws-java-sdk-bundle-1.12.262.jar",
    "jars/jackson-core-2.15.2.jar",
    "jars/jackson-databind-2.15.2.jar",
    "jars/jackson-annotations-2.15.2.jar",
    "jars/slf4j-api-1.7.36.jar",
    "jars/slf4j-reload4j-1.7.36.jar",
    "jars/commons-compress-1.21.jar",
    "jars/avro-1.11.1.jar",
    "jars/aircompressor-0.25.jar",
    "jars/commons-lang3-3.12.0.jar",
    "jars/commons-codec-1.15.jar",
    "jars/commons-text-1.4.jar",
    "jars/commons-io-2.8.0.jar",
    "jars/guava-30.1.1-jre.jar",
    "jars/RoaringBitmap-0.9.47.jar"
]

In [ ]:
conf = (
    pyspark.SparkConf()
        .setAppName('raw_to_standard_etl')
        .set("spark.jars", ",".join(jar_files))
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', nessi_uri)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', aws_s3_endpoint)
        .set('spark.sql.catalog.nessie.warehouse', nessi_warehouse)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key)
        .set('spark.hadoop.fs.s3a.endpoint', aws_s3_endpoint)
        .set('spark.hadoop.fs.s3a.region', aws_s3_region)
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 
             'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider,'
             'org.apache.hadoop.fs.s3a.EnvironmentVariableCredentialsProvider')
)

In [10]:
conf = (
    pyspark.SparkConf()
        .setAppName('raw_to_standard_etl')
        .set('spark.jars.packages', 
             'org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.5.0,'
             'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.4_2.12:0.77.1,'
             'software.amazon.awssdk:bundle:2.24.8,'
             'org.apache.iceberg:iceberg-nessie:1.4.0,'
             'org.apache.hadoop:hadoop-aws:3.3.4,'  
             'org.apache.hadoop:hadoop-common:3.3.4,' 
             'org.apache.iceberg:iceberg-aws-bundle:1.5.2,'
             'io.minio:minio:8.5.0,'
             'org.apache.spark:spark-hadoop-cloud_2.13:3.2.1,'
             'software.amazon.awssdk:url-connection-client:2.24.8,'
             'com.amazonaws:aws-java-sdk-bundle:1.12.262')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', nessi_uri)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', aws_s3_endpoint)
        .set('spark.sql.catalog.nessie.warehouse', nessi_warehouse)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.hadoop.HadoopFileIO')
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key)
        .set('spark.hadoop.fs.s3a.endpoint', aws_s3_endpoint)
        .set('spark.hadoop.fs.s3a.region', aws_s3_region)
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 
             'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider,'
             'org.apache.hadoop.fs.s3a.EnvironmentVariableCredentialsProvider')
)


In [11]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", aws_s3_endpoint)
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")

In [12]:
raw_df = spark.read.csv(raw_csv_path, header=True) 
raw_df.show(50)

+---------+-------+------------+---------+-------+--------------+------+---------+------+--------------+-----------------------+
|ACCOUNTID|BANK_ID|CORE_CUST_ID|   ORGKEY|ORGTYPE|          NAME|GENDER|CUST_TYPE|STATUS|PRIMARY_SOL_ID|RELATIONSHIPOPENINGDATE|
+---------+-------+------------+---------+-------+--------------+------+---------+------+--------------+-----------------------+
|        1|      1|   113391407|113391407|Account|TRAN T *** HAI|     M|    10001| ACTVE|          1404|             23-02-2018|
|        2|      1|   113391457|113391457|Account|TRAN T *** YEN|     F|    10001| ACTVE|          1003|              3/12/2018|
|        3|      1|   113391483|113391483|Account|TRAN T *** BAC|     F|    10001| ACTVE|          2201|             15-06-2018|
|        4|      1|   105260800|105260800|Account|NGUYEN *** UNG|     M|    10001| ACTVE|          1401|             16-11-2018|
|        5|      1|   106573010|106573010|Account|TRAN N *** ANH|     M|    10001| ACTVE|        

In [6]:
spark.sql("CREATE NAMESPACE nessie.lamtest;").show()

++
||
++
++



In [7]:
spark.sql("""CREATE TABLE nessie.lamtest.customer(
            ACCOUNTID STRING,
            BANK_ID STRING,
            CORE_CUST_ID STRING,
            ORGKEY STRING,
            ORGTYPE STRING,
            NAME STRING,
            GENDER STRING,
            CUST_TYPE STRING,
            STATUS STRING,
            PRIMARY_SOL_ID STRING,
            RELATIONSHIPOPENINGDATE DATE)
        USING ICEBERG """)

DataFrame[]